# Evaluate pseudotime goodness-of-fit by scDesign3

In this example, we will show how to use scDesign3 to evaluate the pseudotime goodness-of-fit for different pseudotime labels. If the true labels are unavailable and we have little prior knowledge, the scDesign3 BIC can serve as an unsupervised metric. In this tutorial, we will first use the R package `dyngen` to generate a dataset with ground truth "pseudotime". Then, we will perturb the ground truth pseudotime to worsen its quality and use scDesign3’s BIC to examine pseudotime goodness-of-fit.

## Generation of reference dataset & Simulation

We first generate a synthetic dataset with ground truth pseudotime using `dyngen` package in R. R is called in this jupyter notebook by `rpy2` package.

In [1]:
# pip install rpy2==3.6.1

In [2]:
%load_ext rpy2.ipython

/opt/miniconda3/envs/scdesign/lib/python3.12/site-packages/rpy2/rinterface/__init__.py:1211: UserWarning: Environment variable "PWD" redefined by R and overriding existing variable. Current: "/", R: "/Users/malangtian/Documents/Research_Projects/scDesigner_project/scDesigner/examples"
  warnings.warn(
/opt/miniconda3/envs/scdesign/lib/python3.12/site-packages/rpy2/rinterface/__init__.py:1211: UserWarning: Environment variable "R_SESSION_TMPDIR" redefined by R and overriding existing variable. Current: "/var/folders/5z/8hzvmhln5p36t4tdqc9c52z40000gn/T//RtmpSqMRKc", R: "/var/folders/5z/8hzvmhln5p36t4tdqc9c52z40000gn/T//RtmpnUGBOm"
  warnings.warn(


In [3]:
%%R
library(scDesign3)
library(dyngen)
library(SingleCellExperiment)
library(ggplot2)
library(dplyr)
theme_set(theme_bw())

Registered S3 method overwritten by 'scDesign3':
  method         from  
  predict.gamlss gamlss
Loading required package: SummarizedExperiment
Loading required package: MatrixGenerics
Loading required package: matrixStats

Attaching package: ‘MatrixGenerics’

The following objects are masked from ‘package:matrixStats’:

    colAlls, colAnyNAs, colAnys, colAvgsPerRowSet, colCollapse,
    colCounts, colCummaxs, colCummins, colCumprods, colCumsums,
    colDiffs, colIQRDiffs, colIQRs, colLogSumExps, colMadDiffs,
    colMads, colMaxs, colMeans2, colMedians, colMins, colOrderStats,
    colProds, colQuantiles, colRanges, colRanks, colSdDiffs, colSds,
    colSums2, colTabulates, colVarDiffs, colVars, colWeightedMads,
    colWeightedMeans, colWeightedMedians, colWeightedSds,
    colWeightedVars, rowAlls, rowAnyNAs, rowAnys, rowAvgsPerColSet,
    rowCollapse, rowCounts, rowCummaxs, rowCummins, rowCumprods,
    rowCumsums, rowDiffs, rowIQRDiffs, rowIQRs, rowLogSumExps,
    rowMadDiffs, rowMads, 

In [4]:
%%R
set.seed(123)
backbone <- backbone_linear_simple()
config <-
  initialise_model(
    backbone = backbone,
    num_cells = 500,
    num_tfs = nrow(backbone$module_info),
    num_targets = 100,
    num_hks = 50,
    verbose = FALSE
  )
out <- generate_dataset(
  config,
  format = "sce",
  make_plots = FALSE
  )
example_sce <- out$dataset
colData(example_sce)$pseudotime <- out$model$experiment$cell_info$time 

  |                                                  | 0 % ~calculating   |+                                                 | 1 % ~00s           |++                                                | 3 % ~00s           |++                                                | 4 % ~00s           |+++                                               | 5 % ~00s           |++++                                              | 6 % ~00s           |++++                                              | 8 % ~00s           |+++++                                             | 9 % ~00s           |++++++                                            | 10% ~00s           |++++++                                            | 12% ~00s           |+++++++                                           | 13% ~00s           |++++++++                                          | 14% ~00s           |++++++++                                          | 15% ~00s           |+++++++++                                         | 17% ~00s 

Then we convert the SingleCellExperiment object to an AnnData object in Python.


In [5]:
%%R
library(zellkonverter)
writeH5AD(example_sce, "data/pseudotime_example_sce.h5ad")


    an issue that caused a segfault when used with rpy2:
    https://github.com/rstudio/reticulate/pull/1188
    Make sure that you use a version of that package that includes
    the fix.
    ℹ Using the 'counts' assay as the X matrix


Registered S3 method overwritten by 'zellkonverter':
  method                                             from      
  py_to_r.pandas.core.arrays.categorical.Categorical reticulate


In [6]:
import anndata as ad
example_sce = ad.read_h5ad("data/pseudotime_example_sce.h5ad")
example_sce.X

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 68069 stored elements and shape (500, 155)>

Secondly, we perturb the pseudotime by generating random numbers from uniform distribution and replacing various percentages of the original pseudotime with random numbers. The percentage ranges from 0% to 100%. In the code below, we generate 11 sets of perturbed pseudotime with the percentage of perturbation ranging from 0% to 100%. For each new set of perturbed pseudotime, we create a new SingleCellExperiment object, storing the original count matrix and the corresponding perturbed pseudotime.

In [7]:
example_sce.obs.head()

,step_ix,simulation_i,sim_time,pseudotime
cell1,106,3,16.008333,0.263991
cell2,1169,27,44.008333,0.702819
cell3,1305,30,60.008333,0.988890
cell4,762,18,0.000000,0.027913
cell5,19,1,20.008333,0.383419


In [8]:
import numpy as np
import copy

np.random.seed(123)

original_pseudotime = example_sce.obs["pseudotime"].values

pertubed_sce_list = []

for x in range(10):
    perturb_prop = x/10
    n_cell = int(round(example_sce.n_obs * perturb_prop))
    
    cell_index = np.random.choice(example_sce.n_obs, size=n_cell, replace=False)
    
    new_pseudotime = original_pseudotime.copy()
    new_pseudotime[cell_index] = np.random.uniform(0, 1, n_cell)
    
    new_sce = example_sce.copy()
    new_sce.obs["pseudotime"] = new_pseudotime
    
    pertubed_sce_list.append(new_sce)

Thirdly, we iteratively use scDesign3 to estimate the pseudotime-based simulator.

Original R code:
```{r}
set.seed(123)
scDesign3_result <- lapply(example_sce_list, function(x) {
  res <-  scdesign3(
    sce = x,
    assay_use = "counts",
    celltype = NULL,
    pseudotime = "pseudotime",
    spatial = NULL,
    other_covariates = NULL,
    mu_formula = "s(pseudotime, bs = 'cr', k = 10)",
    sigma_formula = "1",
    corr_formula = "ind",
    copula = "gaussian",
    n_cores = 2
  )
  return(res)
})
```

In [11]:
from scdesigner.simulators import NegBinCopulaSimulator
import scipy

scdesign_result = []
for sce in pertubed_sce_list:
    if scipy.sparse.issparse(sce.X):
        sce.X = sce.X.toarray()
    sim = NegBinCopulaSimulator()
    sim.fit(sce, "~ bs(pseudotime, df=10)")
    scdesign_result.append(sim)

/Users/malangtian/Documents/Research_Projects/scDesigner_project/scDesigner/scdesigner/src/scdesigner/data/group.py:107: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return self.list[idx]
Epoch 1/40:   0%|          | 0/1 [00:00<?, ?it/s]/Users/malangtian/Documents/Research_Projects/scDesigner_project/scDesigner/scdesigner/src/scdesigner/data/group.py:107: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return self.list[idx]
/Users/malangtian/Documents/Research_Projects/scDesigner_project/scDesigner/scdesigner/src/scdesigner/data/group.py:107: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. 